In [2]:
import os
import boto3
import sys
import json
from strands import Agent, tool
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct

# Set AWS region
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

# # Import utils (adjust path as needed)
# Add current directory to Python path explicitly
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"Current directory: {current_dir}")
print(f"Python path includes current dir: {current_dir in sys.path}")

# Now try importing
import techresidential_utils as utils
print("✅ utils imported successfully")

Current directory: /Users/hsin-weilin/Desktop/projects/AWS_AgentCore_TechResidential/agents
Python path includes current dir: True
✅ utils imported successfully


In [3]:
# --- MCP Client Setup for AgentCore Gateway ---

REGION = "us-east-1"
USER_POOL_ID = "us-east-1_VKErYTcMn"
USER_POOL_NAME = "techresidential-agentcore--user-pool"
RESOURCE_SERVER_ID = "techresidential-gateway-id"  
RESOURCE_SERVER_NAME = "techresidential-gateway-name"
CLIENT_NAME = "techresidential-agentcore-gateway-client"

SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]

scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

# Get Gateway URL
# Create an SSM client
ssm = boto3.client('ssm', region_name=REGION)  # Change region if needed
# Retrieve the parameter value
response = ssm.get_parameter(
    Name="AgentCore_Gateway_URL",
    WithDecryption=False  # Set to True if the parameter is a SecureString
)
gateway_url = response['Parameter']['Value']
response2 = ssm.get_parameter(
    Name="NOVA_ACT_API_KEY",
    WithDecryption=True  # Set to True if the parameter is a SecureString
)
nova_act_api_key = response2['Parameter']['Value']

# Get Cognito User Pool Client Secret from Secret Manager
print("Retrieving Cognito User Pool client secret from Secret Manager")
secrets_client = boto3.client('secretsmanager', region_name=REGION)
secret_response = secrets_client.get_secret_value(SecretId='prod/Cognito-Agentcore-Gateway-Client')
secret_dict = json.loads(secret_response['SecretString'])
client_id = secret_dict['client_id']
client_secret = secret_dict['client_secret']

# # Get Client access token from Cognito
print("Requesting access token from Cognito...")
token_response = utils.get_token(USER_POOL_ID, client_id, client_secret, scopeString, REGION)
token = token_response["access_token"]
print("Access token obtained successfully", token)

Retrieving Cognito User Pool client secret from Secret Manager
Requesting access token from Cognito...
Access token obtained successfully eyJraWQiOiJhTUtWVDhINWtHU2JHWlYrMmtTOUt1bEVBVjl3OXpZS0g3VzRuYStPYzB3PSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIzY3I0cm03cmpqOGhnNXN1bWhxa2htMmcxcCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoidGVjaHJlc2lkZW50aWFsLWdhdGV3YXktaWRcL2dhdGV3YXk6d3JpdGUgdGVjaHJlc2lkZW50aWFsLWdhdGV3YXktaWRcL2dhdGV3YXk6cmVhZCIsImF1dGhfdGltZSI6MTc1ODkwNzM2NiwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tXC91cy1lYXN0LTFfVktFcllUY01uIiwiZXhwIjoxNzU4OTEwOTY2LCJpYXQiOjE3NTg5MDczNjYsInZlcnNpb24iOjIsImp0aSI6IjA3Y2EwNTA3LWM5YjItNDQ2OS1hMjc5LWJjNTcwMGMxMjUxOCIsImNsaWVudF9pZCI6IjNjcjRybTdyamo4aGc1c3VtaHFraG0yZzFwIn0.Lxp_dMdZAPRuXM9ocNxyxDS1o2LGaXm6aYP4ixfTQVa9dln-vGrq6y-giSfz2a3y52cbCSKAiW6RyWEBKrfAw-qcEyEuJT_j_W73bOSRYfCO_u-PP6OaSANOTtPxv1rYvIRierMX1CH2W3MM0GqKm7aJVNN7FD8WH3DLjCKcGj-Dtdcar3zGHnjsE4kaTBbQaX0GnEoYRxhdbVastzmP7JoLy2Yi0QIRzFgQgjJ-pnYDgZ4tYJCbRuIvh0l4SOKOaJp9jX

In [4]:
# %%writefile enhanced_strands_claude_with_nova.py
# Example: Enhanced Strands agent with Nova Act browser tool
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
import json
# --- MCP Client Setup for AgentCore Gateway ---
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer
from rich.console import Console
# app = BedrockAgentCoreApp()

# NEW: Nova Act browser tool
@tool
def browse_web(instruction: str, starting_url: str = "https://google.com"):
    """Use Nova Act to browse the web and complete web-based tasks
    
    Args:
        instruction: What you want to accomplish on the web
        starting_url: URL to start from
    
    Returns:
        Result of the web browsing task
    """
    try:
        with browser_session("us-east-1") as browser_client:
            ws_url, headers = browser_client.generate_ws_headers()
            
            try: 
                with NovaAct(
                    cdp_endpoint_url=ws_url,
                    cdp_headers=headers,
                    nova_act_api_key=nova_act_api_key,
                    starting_page=starting_url,
                ) as nova_act:
                    result = nova_act.act(instruction)
                    return f"Web task completed: {str(result)}"
            except Exception as nova_error:
                error_msg = str(nova_error)
                console.print(f"[red]NovaAct error: {error_msg}[/red]")
                
                # Handle specific error cases
                if "credentials are not valid" in error_msg:
                    return "Authentication failed: The provided credentials were rejected by the website."
                elif "captcha" in error_msg.lower():
                    return "Security challenge detected: The website requires human verification."
                elif "timeout" in error_msg.lower():
                    return "Operation timed out: The web task took too long to complete."
                else:
                    return f"Web automation error: {error_msg}"
    except Exception as e:
        return f"Error browsing web: {str(e)}"
print("reset browse_tool function")
# # Initialize enhanced agent with Nova Act
# model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# model = BedrockModel(model_id=model_id)

# agent = Agent(
#     model=model,
#     tools=[calculator, browse_web],  # Now includes web browsing!
#     system_prompt="You're a helpful assistant. You can do math calculations, tell the weather, and browse the web to find information."
# )

# @app.entrypoint
# def enhanced_strands_agent(payload):
#     """Enhanced agent that can browse the web"""
#     user_input = payload.get("prompt")
#     print("User input:", user_input)
#     response = agent(user_input)
#     return response.message['content'][0]['text']

# if __name__ == "__main__":
#     app.run()

reset browse_tool function


In [6]:

@tool
def live_view_with_nova_act(instruction, starting_url):
    try:
        # Step 1: Create browser session
        with browser_session(REGION) as client:
            ws_url, headers = client.generate_ws_headers()

            # Step 2: Start viewer server
            viewer = BrowserViewerServer(client, port=8000)
            viewer_url = viewer.start(open_browser=True)
            print("open browser")
            # Step 4: Use Nova Act to interact with the browser
            try: 
                with NovaAct(
                    cdp_endpoint_url=ws_url,
                    cdp_headers=headers,
                    nova_act_api_key=nova_act_api_key,
                    starting_page=starting_url,
                ) as nova_act:
                    try:
                        result = nova_act.act(instruction)
                        console.print(f"\n[bold green]Nova Act Result:[/bold green] {result}")
                    except nova_act.types.act_errors.ActAgentFailed as auth_error:
                        # Handle authentication or agent errors
                        console.print(f"\n[red]Authentication or agent error:[/red] {str(auth_error)}")
                        return f"Nova Act agent error: {str(auth_error)}"
                    except nova_act.types.act_errors.ActAgentTimedOut as timeout_error:
                        # Handle timeout errors
                        console.print(f"\n[red]Operation timed out:[/red] {str(timeout_error)}")
                        return "The browser automation timed out - the task may be too complex"
                    except Exception as act_error:
                        # Handle other NovaAct execution errors
                        console.print(f"\n[red]Nova Act execution error:[/red] {str(act_error)}")
                        return f"Error during browser automation: {str(act_error)}"
            except Exception as nova_init_error:
                # Handle NovaAct initialization errors
                console.print(f"\n[red]Failed to initialize Nova Act:[/red] {str(nova_init_error)}")
            return f"Could not start browser automation: {str(nova_init_error)}"
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")
    return result

print("live browser tool")

live browser tool


In [ ]:
#Test Nova act only
def create_streamable_http_transport():
    return streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {token}"})

# Create MCP client and discover tools from the gateway using "with" context manager
client = MCPClient(create_streamable_http_transport)

with client:
    gateway_tools = client.list_tools_sync()
    tools = [live_view_with_nova_act]
    # now initialize your agent with these tools
    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    model = BedrockModel(model_id=model_id)
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt="""You're a helpful Web UI automation assistant. Follow this exact process:
        
    login: 
   - Use the live_view_with_nova_act tool with TWO arguments:
     a. instruction: Include BOTH login details AND the task instructions
     b. starting_url: The URL extracted from credentials
   - Format the instruction like: "Login using username: X and password: Y. Then [instructions from prompt file]"
   
IMPORTANT BEHAVIOR RULES:
- Do NOT retry failed tool calls on your own. If a tool fails, explain the error and stop.
- If the web automation tool fails, do not attempt to call it again. Report the error only.
- After any error in web automation, provide a summary of what was completed and what failed.
- Never output the password in plain text in your responses.
- Never make multiple attempts to run the same web automation in a single response.
"""
    )

    print(f"Available tools: {[tool.tool_name for tool in tools]}")

    # Test Invoke Agent
    response = agent("client_name=Resume")
    print(response)


In [10]:
def create_streamable_http_transport():
    return streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {token}"})

# Create MCP client and discover tools from the gateway using "with" context manager
client = MCPClient(create_streamable_http_transport)

with client:
    gateway_tools = client.list_tools_sync()
    tools = gateway_tools + [live_view_with_nova_act]
    # now initialize your agent with these tools
    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    model = BedrockModel(model_id=model_id)
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt="""You're a helpful Web UI automation assistant. Follow this exact process:
        
1. FIRST STAGE - Data Collection:
   - When given a client_name, use the CredentialsRetriever___get_credentials tool to get login credentials
   - Then use the PromptFileRetriever___get_prompt_file tool to get the task instructions
   - Extract and clearly identify: website URL, username (could be email or something else), password, and task instructions

2. SECOND STAGE - Web Automation:
   - Use the live_view_with_nova_act tool with TWO arguments:
     a. instruction: Include BOTH login details AND the task instructions
     b. starting_url: The URL extracted from credentials
   - Format the instruction like: "Login using username: X and password: Y. Then [instructions from prompt file]"
   
IMPORTANT BEHAVIOR RULES:
- Do NOT retry failed tool calls on your own. If a tool fails, explain the error and stop.
- If the web automation tool fails, do not attempt to call it again. Report the error only.
- After any error in web automation, provide a summary of what was completed and what failed.
- Never output the password in plain text in your responses.
- Never make multiple attempts to run the same web automation in a single response.
"""
    )

    print(f"Available tools: {[tool.tool_name for tool in tools]}")

    # Test Invoke Agent
    response = agent("client_name=Resume")
    print(response)


Available tools: ['CredentialsRetriever___get_credentials', 'PromptFileRetriever___get_prompt_file', 'live_view_with_nova_act']
I'll help you automate tasks for the client "Resume". First, I need to gather the necessary credentials and instructions.

Let me retrieve the login credentials and website information:
Tool #1: CredentialsRetriever___get_credentials
Now, let me get the task instructions:
Tool #2: PromptFileRetriever___get_prompt_file
I've gathered all the necessary information:

Website URL: https://www.myperfectresume.com/signin
Username: hsinweilin@arizona.edu
Password: [REDACTED for security]
Task Instructions: "After you sign in, on the top left corner, there is my most updated resume, it should called something like Hsinwei_Lin_Resume. There is a download button. Click that download button."

Now I'll use the web automation tool to complete this task:
Tool #3: live_view_with_nova_act


✅ DCV SDK found (1,938,365 bytes)

ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 8000): address already in use


✅ Viewer server running at: http://localhost:8000

Check browser console (F12) for detailed debug information

Opening browser...

open browser


2025-09-26 10:28:49,176 - nova_act.impl.playwright - ERROR - Failed to start and initialize Playwright for NovaAct: Page.goto: net::ERR_HTTP2_PROTOCOL_ERROR at https://www.myperfectresume.com/signin
Call log:
  - navigating to "https://www.myperfectresume.com/signin", waiting until "load"
Traceback (most recent call last):
  File "/Users/hsin-weilin/.cache/uv/builds-v0/.tmpgPnxbC/lib/python3.11/site-packages/nova_act/impl/playwright.py", line 279, in start
    self._init_browser_context(context, trusted_page)
  File "/Users/hsin-weilin/.cache/uv/builds-v0/.tmpgPnxbC/lib/python3.11/site-packages/nova_act/impl/playwright.py", line 92, in _init_browser_context
    first_page.goto(self._starting_page)
  File "/Users/hsin-weilin/.cache/uv/builds-v0/.tmpgPnxbC/lib/python3.11/site-packages/playwright/sync_api/_generated.py", line 9019, in goto
    self._sync(
  File "/Users/hsin-weilin/.cache/uv/builds-v0/.tmpgPnxbC/lib/python3.11/site-packages/playwright/_impl/_sync_base.py", line 115, in _s

I apologize for the error that occurred during the web automation process. The tool encountered a technical issue (a NameError indicating "console" is not defined), which prevented the automation from being completed successfully.

Here's a summary of what was attempted:
- I retrieved the credentials and instructions for accessing your resume on myperfectresume.com
- The web automation was initiated but encountered a backend technical error before any actions could be performed

Unfortunately, I cannot retry the automation tool on my own as that would violate my behavior rules. The error appears to be a technical issue with the automation system rather than with the instructions or credentials provided.

Would you like me to try a different approach or would you prefer to report this technical issue to support?I apologize for the error that occurred during the web automation process. The tool encountered a technical issue (a NameError indicating "console" is not defined), which prevent